In [9]:
import pandas as pd
from datetime import datetime
import datetime

In [10]:
payments = pd.read_excel('./data_1.xlsx')
payments

,ЛС,Сумма платежа,Дата загрузки,Источник платежа,способ оплаты,Комментарий к платежу
0,1,4.80,2019-04-16,Ручной ввод,Офис,NaN
1,1,14.40,2018-10-17,Ввод из реестра,Сбербанк России,NaN
2,1,14.40,2019-01-21,Ввод из реестра,КС банк,NaN
3,1,32.40,2020-12-11,Ввод из реестра,КС банк,NaN
4,1,56.40,2018-02-09,Ввод из реестра,КС банк,NaN
...,...,...,...,...,...,...
356841,49148,31.20,2021-04-09,Ввод из реестра,Сбербанк России,NaN
356842,49149,31.20,2021-04-27,Ввод из реестра,КС банк,NaN
356843,49150,25.81,2021-06-10,Ручной ввод,Выписки,NaN
356844,49151,20.63,2021-04-16,Ввод из реестра,Сбербанк России,NaN


In [11]:
accounts = pd.read_excel('./data_2.xlsx')
accounts

,ИД дома,ИД подъезда,ИД квартиры,Лицевой счет (ЛС),Услуга,Ставка тарифа,Статус услуги,Дата начала услуги
0,100,1624,43036,43036,ТО МКД,5.2,Активна,2008-01-23
1,100,1624,43037,43037,ТО МКД,5.4,Активна,2008-01-23
2,100,1624,43038,43038,ТО МКД,5.2,Активна,2008-01-23
3,100,1624,43039,43039,ТО МКД,5.2,Активна,2008-01-23
4,100,1624,43040,43040,ТО МКД,5.2,Активна,2008-01-23
...,...,...,...,...,...,...,...,...
49538,5219,1884,49529,49069,ТО МКД,5.4,Активна,2019-03-28
49539,5219,1884,49530,49070,ТО МКД,5.2,Активна,2019-03-28
49540,5219,1884,49531,49071,ТО МКД,5.4,Активна,2019-03-28
49541,5219,1884,49532,49072,ТО МКД,5.4,Активна,2019-03-28


In [12]:
accounts['Услуга'].unique()

array(['ТО МКД', 'ВБ', 'ТО Дверь', 'Рассрочка за Монтаж ВБ', 'ТО СВОД',
       'Рассрочка за Апгрейд ВБ'], dtype=object)

In [13]:
#смотрим данные таблицы
accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49543 entries, 0 to 49542
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ИД дома             49543 non-null  int64         
 1   ИД подъезда         49543 non-null  int64         
 2   ИД квартиры         49543 non-null  int64         
 3   Лицевой счет (ЛС)   49543 non-null  int64         
 4   Услуга              49543 non-null  object        
 5   Ставка тарифа       49316 non-null  float64       
 6   Статус услуги       49543 non-null  object        
 7   Дата начала услуги  49543 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 3.0+ MB


Суть проверки:
проверяем, что:
"Одновременно на одной квартире может быть только одна базовая услуга ТО МКД, ТО СВОД или ТО ДВЕРЬ в статусе != Завершена."

In [14]:
selected_services = ['ТО МКД', 'ТО СВОД', 'ТО Дверь']

many_services_1 =accounts.query('`Услуга` in @selected_services')\
                         .query('`Статус услуги` != "Завершена"')\
                         .groupby(['ИД квартиры'])\
                         .agg({'ИД квартиры': 'count'})\
                         .query('`ИД квартиры` > 1')

#Результат проверки: Существуют номера квартир, в которых условие не выполняется:
many_services_1

,ИД квартиры
ИД квартиры,
2506,2
22730,2
23262,2
40749,2
43410,2
43942,2
47526,2
48383,2
49522,2



Предпринятый или предлагаемый способ устранение дефекта данных:
1. Уточнить, являются ли эти статусы приемлимыми, потому что по логике, они вполне реальные
2. Если это всё-таки недопустимые статусы, то удалить строки с неприемлимыми статусами из таблицы
3. Сейчас устранять ничего не буду.

Суть проверки: Если на одном ЛС есть несколько базовых услуг (ТО МКД, ТО СВОД или ТО ДВЕРЬ), то ранее действующая должна быть в статусе Завершена.


In [15]:
selected_services = ['ТО МКД', 'ТО СВОД', 'ТО Дверь']

many_services_2 = accounts.query('`Услуга` in @selected_services')\
                          .groupby(['Лицевой счет (ЛС)'], as_index=False)\
                          .agg({'Услуга':'count'})\
                          .query('`Услуга` > 1')

acc = many_services_2['Лицевой счет (ЛС)']

In [16]:
max_date = accounts.query('`Лицевой счет (ЛС)` in @acc')\
                   .query('`Услуга` in @selected_services')\
                   .groupby('Лицевой счет (ЛС)', as_index=False)\
                   .agg({'Дата начала услуги': 'max'})
max_date

,Лицевой счет (ЛС),Дата начала услуги
0,1,2021-03-01
1,22,2021-03-29
2,26,2019-02-02
3,84,2020-12-24
4,208,2019-04-03
...,...,...
5547,49134,2020-11-01
5548,49136,2020-12-01
5549,49140,2021-05-20
5550,49150,2021-02-02


In [17]:
min_date = accounts.query('`Лицевой счет (ЛС)` in @acc')\
                   .query('`Услуга` in @selected_services')\
                   .query('`Статус услуги` != "Завершена"')\
                   .groupby('Лицевой счет (ЛС)', as_index=False)\
                   .agg({'Дата начала услуги': 'min'})

min_date

,Лицевой счет (ЛС),Дата начала услуги
0,1,2021-03-01
1,22,2021-03-29
2,26,2019-02-02
3,84,2020-12-24
4,208,2019-04-03
...,...,...
5547,49134,2020-11-01
5548,49136,2020-12-01
5549,49140,2021-05-20
5550,49150,2021-02-02


In [18]:
#Результат проверки: Существуют ЛС, для которых условие не выполняется:
max_date.join(min_date, lsuffix=" max", rsuffix=" min").query('`Дата начала услуги max` > `Дата начала услуги min`')

,Лицевой счет (ЛС) max,Дата начала услуги max,Лицевой счет (ЛС) min,Дата начала услуги min
418,2506,2021-11-01,2506,2017-11-14
3652,22730,2019-01-01,22730,2007-02-25
3665,23262,2021-08-19,23262,2019-01-29
4690,40749,2021-07-01,40749,2007-11-12
4750,43410,2019-11-01,43410,2008-02-26
4770,43942,2019-08-01,43942,2008-04-11
5409,47612,2020-01-01,47612,2010-12-22
5434,48140,2019-10-24,48140,2014-07-01
5539,49062,2019-03-28,49062,2019-03-01


Предпринятый или предлагаемый способ устранение дефекта данных:

Уточнить, являются ли эти статусы приемлимыми, потому что по логике, они вполне реальные.

Если это всё-таки недопустимые статусы, то удалить строки с неприемлимыми статусами из таблицы.

Сейчас устранять ничего не буду.

2. Рассчитать для каждого ЛС сумму всех поступивших платежей с учетом возвратов, но исключая списания.

In [19]:
#вспомним нашу таблицу
payments.head()

,ЛС,Сумма платежа,Дата загрузки,Источник платежа,способ оплаты,Комментарий к платежу
0,1,4.8,2019-04-16,Ручной ввод,Офис,NaN
1,1,14.4,2018-10-17,Ввод из реестра,Сбербанк России,NaN
2,1,14.4,2019-01-21,Ввод из реестра,КС банк,NaN
3,1,32.4,2020-12-11,Ввод из реестра,КС банк,NaN
4,1,56.4,2018-02-09,Ввод из реестра,КС банк,NaN


In [20]:
#проверим, какие вообще источники платежа есть
payments['Источник платежа'].unique()

array(['Ручной ввод', 'Ввод из реестра', 'Возврат', 'Сторно минус',
       'Авто. списание', 'Возврат списания', 'Ручное списание',
       'Сторно плюс'], dtype=object)

In [21]:
#создадим таблицу только с нужными нам платежами
#не очень понятно, нужно ли исключать "Возврат списания", ориентируясь на свою логику - я оставила
pays = payments[(payments['Источник платежа'] != 'Авто. списание') & (payments['Источник платежа'] != 'Ручное списание')]
pays['Источник платежа'].unique()

array(['Ручной ввод', 'Ввод из реестра', 'Возврат', 'Сторно минус',
       'Возврат списания', 'Сторно плюс'], dtype=object)

In [22]:
#итог: ответ задачи
task_2 = pays.groupby('ЛС', as_index=False).agg({'Сумма платежа': 'sum'})
task_2

,ЛС,Сумма платежа
0,1,182.40
1,2,236.40
2,3,230.40
3,4,236.40
4,5,222.90
...,...,...
39626,49146,62.40
39627,49148,31.20
39628,49149,31.20
39629,49150,25.81


3. Рассчитать для каждого ЛС плановую выручку с учетом дат начала услуг на ЛС и ставки тарифа

In [23]:
status = ['Активна','Завершена']

tariff = accounts[['Лицевой счет (ЛС)','Дата начала услуги','Ставка тарифа','Статус услуги']]\
         .query('`Статус услуги` in @status')\
         .sort_values(['Лицевой счет (ЛС)','Дата начала услуги'])
tariff

,Лицевой счет (ЛС),Дата начала услуги,Ставка тарифа,Статус услуги
13170,1,2006-09-02,5.4,Завершена
13172,1,2020-01-01,5.0,Завершена
13171,1,2021-03-01,5.2,Активна
13173,2,2007-10-01,5.2,Активна
13174,3,2006-09-02,5.4,Активна
...,...,...,...,...
25445,49150,2021-02-02,5.4,Активна
7524,49151,2006-12-23,5.4,Завершена
7525,49151,2021-03-02,5.4,Активна
36890,49152,2021-08-13,5.4,Активна


In [24]:
tariff['Дата окончания услуги'] = tariff.groupby('Лицевой счет (ЛС)')['Дата начала услуги'].shift(-1)
tariff['Дата окончания услуги'] = tariff['Дата окончания услуги'] - pd.Timedelta(days=1)
tariff['Дата окончания услуги'] = tariff['Дата окончания услуги'].fillna('2021-10-13')

tariff
#df['prev_value'] = df.groupby('object')['value'].shift()

,Лицевой счет (ЛС),Дата начала услуги,Ставка тарифа,Статус услуги,Дата окончания услуги
13170,1,2006-09-02,5.4,Завершена,2019-12-31
13172,1,2020-01-01,5.0,Завершена,2021-02-28
13171,1,2021-03-01,5.2,Активна,2021-10-13
13173,2,2007-10-01,5.2,Активна,2021-10-13
13174,3,2006-09-02,5.4,Активна,2021-10-13
...,...,...,...,...,...
25445,49150,2021-02-02,5.4,Активна,2021-10-13
7524,49151,2006-12-23,5.4,Завершена,2021-03-01
7525,49151,2021-03-02,5.4,Активна,2021-10-13
36890,49152,2021-08-13,5.4,Активна,2021-10-13


In [25]:
tariff = tariff.query('`Дата окончания услуги`>"2018-01-01"')
tariff['Дата начала услуги'] = tariff['Дата начала услуги'].mask(tariff['Дата начала услуги']<'2018-01-01', '2018-01-01')
tariff

/var/folders/ww/3csmp5ln1b9cpttvw31wchgr0000gp/T/ipykernel_5113/3213401335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tariff['Дата начала услуги'] = tariff['Дата начала услуги'].mask(tariff['Дата начала услуги']<'2018-01-01', '2018-01-01')


,Лицевой счет (ЛС),Дата начала услуги,Ставка тарифа,Статус услуги,Дата окончания услуги
13170,1,2018-01-01,5.4,Завершена,2019-12-31
13172,1,2020-01-01,5.0,Завершена,2021-02-28
13171,1,2021-03-01,5.2,Активна,2021-10-13
13173,2,2018-01-01,5.2,Активна,2021-10-13
13174,3,2018-01-01,5.4,Активна,2021-10-13
...,...,...,...,...,...
25445,49150,2021-02-02,5.4,Активна,2021-10-13
7524,49151,2018-01-01,5.4,Завершена,2021-03-01
7525,49151,2021-03-02,5.4,Активна,2021-10-13
36890,49152,2021-08-13,5.4,Активна,2021-10-13


In [26]:
tariff['Количество месяцев'] = (tariff['Дата окончания услуги'].dt.to_period('M').astype(int) - tariff['Дата начала услуги'].dt.to_period('M').astype(int))
tariff


/var/folders/ww/3csmp5ln1b9cpttvw31wchgr0000gp/T/ipykernel_5113/723596654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tariff['Количество месяцев'] = (tariff['Дата окончания услуги'].dt.to_period('M').astype(int) - tariff['Дата начала услуги'].dt.to_period('M').astype(int))


,Лицевой счет (ЛС),Дата начала услуги,Ставка тарифа,Статус услуги,Дата окончания услуги,Количество месяцев
13170,1,2018-01-01,5.4,Завершена,2019-12-31,23
13172,1,2020-01-01,5.0,Завершена,2021-02-28,13
13171,1,2021-03-01,5.2,Активна,2021-10-13,7
13173,2,2018-01-01,5.2,Активна,2021-10-13,45
13174,3,2018-01-01,5.4,Активна,2021-10-13,45
...,...,...,...,...,...,...
25445,49150,2021-02-02,5.4,Активна,2021-10-13,8
7524,49151,2018-01-01,5.4,Завершена,2021-03-01,38
7525,49151,2021-03-02,5.4,Активна,2021-10-13,7
36890,49152,2021-08-13,5.4,Активна,2021-10-13,2


In [27]:
tariff['Выручка за период'] = tariff['Ставка тарифа'] * tariff['Количество месяцев']
tariff

/var/folders/ww/3csmp5ln1b9cpttvw31wchgr0000gp/T/ipykernel_5113/2914865179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tariff['Выручка за период'] = tariff['Ставка тарифа'] * tariff['Количество месяцев']


,Лицевой счет (ЛС),Дата начала услуги,Ставка тарифа,Статус услуги,Дата окончания услуги,Количество месяцев,Выручка за период
13170,1,2018-01-01,5.4,Завершена,2019-12-31,23,124.2
13172,1,2020-01-01,5.0,Завершена,2021-02-28,13,65.0
13171,1,2021-03-01,5.2,Активна,2021-10-13,7,36.4
13173,2,2018-01-01,5.2,Активна,2021-10-13,45,234.0
13174,3,2018-01-01,5.4,Активна,2021-10-13,45,243.0
...,...,...,...,...,...,...,...
25445,49150,2021-02-02,5.4,Активна,2021-10-13,8,43.2
7524,49151,2018-01-01,5.4,Завершена,2021-03-01,38,205.2
7525,49151,2021-03-02,5.4,Активна,2021-10-13,7,37.8
36890,49152,2021-08-13,5.4,Активна,2021-10-13,2,10.8


In [28]:
task_3 = tariff.groupby('Лицевой счет (ЛС)', as_index=False).agg({'Выручка за период': 'sum'})
task_3

,Лицевой счет (ЛС),Выручка за период
0,1,225.6
1,2,234.0
2,3,243.0
3,4,234.0
4,5,243.0
...,...,...
38688,49149,54.0
38689,49150,243.0
38690,49151,243.0
38691,49152,10.8


4. Рассчитать для каждого ЛС сальдо: плановая выручка (п. 3) минус сумма платежей (из п. 2).

In [29]:
Payments_tariff = task_2.merge(task_3, left_on='ЛС', right_on='Лицевой счет (ЛС)')
Payments_tariff['Сальдо'] = Payments_tariff['Выручка за период'] - Payments_tariff['Сумма платежа']
Payments_tariff

,ЛС,Сумма платежа,Лицевой счет (ЛС),Выручка за период,Сальдо
0,1,182.40,1,225.6,43.20
1,2,236.40,2,234.0,-2.40
2,3,230.40,3,243.0,12.60
3,4,236.40,4,234.0,-2.40
4,5,222.90,5,243.0,20.10
...,...,...,...,...,...
38294,49146,62.40,49146,52.0,-10.40
38295,49148,31.20,49148,54.0,22.80
38296,49149,31.20,49149,54.0,22.80
38297,49150,25.81,49150,243.0,217.19


In [30]:
task_4 = Payments_tariff[['Лицевой счет (ЛС)','Сальдо']]
task_4

,Лицевой счет (ЛС),Сальдо
0,1,43.20
1,2,-2.40
2,3,12.60
3,4,-2.40
4,5,20.10
...,...,...
38294,49146,-10.40
38295,49148,22.80
38296,49149,22.80
38297,49150,217.19


5. Рассчитать для каждой квартиры частоту платежей

In [31]:
payments = payments.rename(columns={'ЛС':'Лицевой счет (ЛС)'})
payments

,Лицевой счет (ЛС),Сумма платежа,Дата загрузки,Источник платежа,способ оплаты,Комментарий к платежу
0,1,4.80,2019-04-16,Ручной ввод,Офис,NaN
1,1,14.40,2018-10-17,Ввод из реестра,Сбербанк России,NaN
2,1,14.40,2019-01-21,Ввод из реестра,КС банк,NaN
3,1,32.40,2020-12-11,Ввод из реестра,КС банк,NaN
4,1,56.40,2018-02-09,Ввод из реестра,КС банк,NaN
...,...,...,...,...,...,...
356841,49148,31.20,2021-04-09,Ввод из реестра,Сбербанк России,NaN
356842,49149,31.20,2021-04-27,Ввод из реестра,КС банк,NaN
356843,49150,25.81,2021-06-10,Ручной ввод,Выписки,NaN
356844,49151,20.63,2021-04-16,Ввод из реестра,Сбербанк России,NaN


In [37]:
payments_flat = payments.merge(accounts[['Лицевой счет (ЛС)','ИД квартиры']], on='Лицевой счет (ЛС)' )
task_5 = payments_flat.groupby('ИД квартиры', as_index=False).agg({'Сумма платежа': 'count'}).rename(columns={'Сумма платежа':'Количество платежей'})
task_5

,ИД квартиры,Количество платежей
0,1,18
1,2,100
2,3,37
3,4,4
4,5,61
...,...,...
41377,49547,1
41378,49548,9
41379,49550,1
41380,49552,1
